In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import itertools
import copy
import pandas as pd
import matplotlib.ticker as ticker
import pickle
import matplotlib as mpl

In [ ]:
# Class to do heritability estimation
class heritability :
    def __init__(self, maf, n_indiv, n_marker, hsq) :
        self.maf = maf
        self.n_indiv = n_indiv
        self.n_marker = n_marker
        self.hsq = hsq
        self.genotypes = self.get_genotypes()
        self.genotypes_norm = self.normalize_genotypes()
        self.grm = self.get_grm()
        self.noncausal_linked_set = False
    
    # Returns the genotypes for this class. Each row is an individual
    # And each column is a marker. I.e. there are n_indiv rows and n_marker columns
    def get_genotypes(self) :
        if len(self.maf) > 1 and len(self.maf) != self.n_marker :
            raise Exception("Heritability class (get_genotypes): len(self.maf) != self.n_marker")
        return np.random.binomial(2, self.maf, size = (self.n_indiv, self.n_marker))

    # Normalizes the genotypes according to (x-2p)/sqrt(2p(1-p))
    def normalize_genotypes(self) :
        genotypes = self.genotypes
        genotypes_norm = np.zeros(self.genotypes.shape)
        for i in range(genotypes.shape[1]) :
            p = np.sum(genotypes[:,i])/(genotypes.shape[0] * 2)
            genotypes_norm[:,i] = (genotypes[:,i] - 2*p)/np.sqrt(2 * p * (1-p))
        return genotypes_norm
    
    # Calculates the GRM according to XX^T 
    def get_grm(self) :
        return self.genotypes_norm @ self.genotypes_norm.transpose() / self.n_marker
    
    # Calculates the GRM but only uses the first num_marker markers
    def set_sub_grm(self, num_marker) :
        tmp = self.genotypes_norm[:,0:num_marker]
        self.grm = tmp @ tmp.transpose() / num_marker
    
    # This calculates genotypes based off of ALL genotypes in self.genotypes_norm
    # This means that if you call duplicate_genotypes or noncausal_genotypes, and then later
    # Call get_phenotypes, it will use all of the duplicates/noncausals
    def get_phenotypes(self) :
        n_causal = int(self.n_marker)
        betas = np.random.normal(0, np.sqrt(self.hsq/n_causal), n_causal)
        phen = self.genotypes_norm.dot(betas) + np.random.normal(0, np.sqrt(1-self.hsq), self.n_indiv)
        return phen
     
    # Replaces each genotype of genotypes with a probability replace_prob
    # Goes through each row and rolls a bernoulli for each marker
    def replace_with_duplicates(self, genotypes, replace_prob) :
        # Go through every marker
        for i in range(genotypes.shape[0]):
            for j in range(genotypes.shape[1]) :
                # Check if we should replace this marker
                if np.random.uniform() < replace_prob :
                    genotypes[i, j] = np.random.binomial(2, self.maf)
    
    # m_dup is the number of markers that are being duplicated. Always duplicates the first m_dup
    # n_dup is the number of times that those m_dup markers are duplicated
    # Also updates GRM 
    def set_duplicate_genotypes(self, m_dup, n_dup, replace_prob = 0) :
        dup_genotypes = self.genotypes[:,0:m_dup]
        rep_genotypes = np.repeat(dup_genotypes, n_dup, 1)
        self.replace_with_duplicates(rep_genotypes, replace_prob)
        new_genotypes = np.append(self.genotypes, rep_genotypes , 1)
        self.genotypes = new_genotypes
        self.genotypes_norm = self.normalize_genotypes()
        self.n_marker = self.genotypes.shape[1]
        self.grm = self.get_grm()
        
    # This adds new noncausal markers to the dataset. Adds num_noncausal markers to each indiv
    # These are unlinked
    def add_noncausal_genotypes(self, num_noncausal) :
        noncausal_add = np.random.binomial(2, self.maf, size = (self.n_indiv, num_noncausal))
        new_genotypes = np.append(self.genotypes, noncausal_add, 1)
        self.genotypes = new_genotypes
        self.genotypes_norm = self.normalize_genotypes()
        self.grm = self.get_grm()
        self.n_marker = self.genotypes.shape[1]
    
    # Adds new markers. The first time you call this function, it will add num_markers noncausal markers
    # To the genotypes and recalculate the grm. 
    # Consecutive calls will copy the last num_markers and append them to the end of genotypes
    def add_noncausal_linked_genotypes(self, num_markers) :
        if self.noncausal_linked_set == False :
            self.add_noncausal_genotypes(num_markers)
            self.noncausal_linked_set = True
        else :
            # This bottom line of code is a little weird but the first colon means we take all rows
            # The second -num_markers: means we take the last num_markers columns
            dup_genotypes = self.genotypes[:,-num_markers:]
            new_genotypes = np.append(self.genotypes, np.repeat(dup_genotypes, 1, 1), 1)
            self.genotypes = new_genotypes
            self.genotypes_norm = self.normalize_genotypes()
            self.grm = self.get_grm()
            self.n_marker = self.genotypes.shape[1]
        
    
# Likelihood for sigma = [sigma_g^2, sigma_e^2]
def ll(sigma, grm, phenotypes) :
    sigma_g = sigma[0]
    sigma_e = sigma[1]
    n_indiv = grm.shape[0]
    V = sigma_g * grm + sigma_e * np.identity(n_indiv)
    s, logdet = np.linalg.slogdet(V)
    return -n_indiv/2 * np.log(2 * np.pi) - 1/2 * (s * logdet + phenotypes.transpose() @ np.linalg.inv(V) @ phenotypes)
    

    
maf = .2
n_indiv = 1000
n_marker = 200



# Changing the number of individuals

In [ ]:
afr_maf = pd.read_csv("markers1shuffle.txt", delimiter = " ")["afr_alt"].to_numpy()

# Use the same phenotypes and genotypes, but continue to duplicate the genotypes
# Here we are duplicating the first 40 genotypes up to 8 times. The duplicated genotypes are causal.
f, axs = plt.subplots(4, 2)
f.set_size_inches(16, 8)
f.subplots_adjust(hspace = .4, wspace = .2)
axs = axs.ravel()
xs = []
ys = []
zs = []
i = 1
n_iter = 10
index = 0
x,y = np.meshgrid(np.arange(.01, 1, step = .3), np.arange(0.01, 1, step = .1))
for n_people, n_markers in [[1000, 200], [200, 1000], [200, 3000], [2000, 1000]]:
    print(n_people, n_markers)
    for repeats in [0, int(n_markers * .1)]:
        z_iter = []
        for it in range(n_iter) :
            z = np.zeros(x.shape)
            h = heritability(afr_maf[:n_markers], n_people, n_markers, .8)
            phenotypes = h.get_phenotypes()
            if repeats != 0:
                h.set_duplicate_genotypes(repeats, 8)
            x,y = np.meshgrid(np.arange(.01, 1, step = .3), np.arange(0.01, 1, step = .1))
            for i in range(x.shape[0]) :
                for j in range(x.shape[1]) :
                    l = ll([x[i,j],y[i,j]], h.grm, phenotypes)
                    if l < -120 :
                        z[i,j] = l#-120
                    else :
                        z[i,j] = l
            z_iter.append(z)
        z = sum(z_iter)/n_iter
        xs.append(x)
        ys.append(y)
        zs.append(z)
        cf = axs[index].contourf(x,y,z)
        f.colorbar(cf, ax = axs[index])
        axs[index].set_xlabel("Sigma_g")
        axs[index].set_ylabel("Sigma_e")
        axs[index].set_title(f"{n_people} Repeats")
        index += 1
f.suptitle("Duplicated (40) Causal Genotypes", size = 20, y = .95)



In [ ]:
pickle.dump(xs, open("x.pickle", "wb"))
pickle.dump(ys, open("y.pickle", "wb"))
pickle.dump(zs, open("z.pickle", "wb"))

In [ ]:
#Thing to format colorbar
def myfmt(x, pos):
    return '{0:.3f}'.format(x)

cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=-.2, vmax=0)

zs = pickle.load(open("z.pickle", "rb"))
xs = pickle.load(open("x.pickle", "rb"))
ys = pickle.load(open("y.pickle", "rb"))
f, axs = plt.subplots(5,2, figsize=[10,12])
# f.set_size(12, 12)
f.subplots_adjust(hspace = .4, wspace = .2)
axs = axs.ravel()
titles = ["A", "B", "C", "D", "E", "F", "G", "H"]
index = 0 
n1000q = np.quantile(zs[1]/1000 - max(zs[1].flatten())/1000, .6)
n200q = np.quantile(zs[2]/200 - max(zs[2].flatten())/200, .6)
for n_people, n_markers in [[1000, 200], [200, 1000], [200, 3000], [2000, 1000]]:
    for repeats in [0, int(n_markers * .1)]:
        znew = copy.deepcopy(zs[index])
        znew = znew/(n_people)
        znew = znew - max(znew.flatten())
        if n_people >= 1000:
            znew[znew < n1000q] = n1000q
            cf = axs[index].contourf(xs[index], ys[index], znew,  levels=np.linspace(-.2, 0, 41))
        else:
            znew[znew < n200q] = n200q
            cf = axs[index].contourf(xs[index], ys[index], znew,  levels=np.linspace(-0.036, 0, 41))
        max_x = xs[1][znew == znew.max()]
        max_y = ys[1][znew == znew.max()]
        axs[index].scatter(max_x, max_y, c="red")
        if index in [1, 3, 5, 7]:
            axs[index].get_yaxis().set_visible(False)
        if index in [0, 1, 2, 3, 4, 5]:
            axs[index].get_xaxis().set_visible(False)
        if index == 0 :
            axs[index].set_ylabel("(i)")
        if index == 2 :
            axs[index].set_ylabel("(ii)")
        if index == 4 :
            axs[index].set_ylabel("(iii)")
        if index == 6 :
            axs[index].set_ylabel("(iv)")
        if index == 0 :
            axs[index].set_title("A")
        if index == 1:
            axs[index].set_title("B")
        index += 1 

cf = axs[index].contourf(xs[0], ys[0], znew,  levels=np.linspace(-.2, 0, 41))
axs[index].set_visible(False)
f.colorbar(cf, ax=axs[index], format=ticker.FuncFormatter(myfmt), location="top")
index += 1
cf = axs[index].contourf(xs[0], ys[0], znew,  levels=np.linspace(-.036, 0, 41))
axs[index].set_visible(False)
f.colorbar(cf, ax=axs[index], format=ticker.FuncFormatter(myfmt), location="top")
f.supxlabel(r'$\sigma_g^2$')
f.supylabel(r'$\sigma_e^2$')
plt.subplots_adjust(wspace=0, hspace=0)
f.tight_layout()
f.savefig("maxlike_repeat_params.png", dpi=300, bbox_inches="tight")
